In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/connector"

In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/password_manager"

In [0]:
conn, success, message = snowflake_connector("ashwin009", get_password('snf_conn'), "WSYCJNH-JF51761",'COMPUTE_WH','INSHORTS','ETL')

In [0]:
cursor = conn.cursor()

In [0]:

cursor.execute('Truncate table foundation.user_retention_details')
print(cursor.fetchall())
cursor.execute(f'''INSERT INTO foundation.user_retention_details
SELECT 
    u.deviceid, 
    u.install_dt, 
    retention_period,
    CASE WHEN e.event_date IS NOT NULL THEN TRUE ELSE FALSE END AS retained
FROM (
    SELECT deviceid, install_dt, 'D1' AS retention_period, DATEADD(DAY, 1, install_dt) AS target_date FROM ETL.user
    UNION ALL
    SELECT deviceid, install_dt, 'D2' AS retention_period, DATEADD(DAY, 2, install_dt) AS target_date FROM ETL.user
    UNION ALL
    SELECT deviceid, install_dt, 'W1' AS retention_period, DATEADD(DAY, 7, install_dt) AS target_date FROM ETL.user
    UNION ALL
    SELECT deviceid, install_dt, 'M1' AS retention_period, DATEADD(MONTH, 1, install_dt) AS target_date FROM ETL.user
    UNION ALL
    SELECT deviceid, install_dt, 'M3' AS retention_period, DATEADD(MONTH, 3, install_dt) AS target_date FROM ETL.user
) u
LEFT JOIN (
    SELECT deviceid, DATE(eventtimestamp) AS event_date
    FROM ETL.event
) e ON u.deviceid = e.deviceid AND u.target_date = e.event_date;''')
print(cursor.fetchall())